In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import eda
import present_value as PresentValue

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


c:\Users\forer\anaconda3\envs\practicum\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
filename = "../data/BASE DE DATOS PRESUPUESTOS.xlsx"
preproccesing = eda.EDA(filename)
df = preproccesing.assemble_project()
df

,NOMBRE DEL PROYECTO,CÓDIGO DEL PROYECTO,AÑO INICIO,FASE,DEPARTAMENTO,LONGITUD KM,PUENTES VEHICULARES UND,PUENTES VEHICULARES M2,PUENTES PEATONALES UND,PUENTES PEATONALES M2,...,7 - SOCAVACIÓN,8 - ESTRUCTURAS,9 - TÚNELES,10 - URBANISMO Y PAISAJISMO,11 - PREDIAL,12 - IMPACTO AMBIENTAL,13 - CANTIDADES,14 - EVALUACIÓN SOCIOECONÓMICA,15 - OTROS - MANEJO DE REDES,16 - DIRECCIÓN Y COORDINACIÓN
0,PEDREGAL - PASTO UF4-UF5,6935,2016,Fase III - Diseños a detalle,Nariño,15.760,4.0,6292.00000,0.0,0.000,...,7.078101e+07,2.261120e+08,0.0,1.612921e+07,0.0,0.00,0.000000e+00,0.0,1.171310e+08,1.042543e+08
1,PEDREGAL - PASTO UF4-UF5,6935,2016,Fase III - Diseños a detalle,Nariño,22.200,0.0,0.00000,1.0,77.000,...,7.078101e+07,2.261120e+08,0.0,1.612921e+07,0.0,0.00,0.000000e+00,0.0,1.171310e+08,1.042543e+08
2,BUGA - BUENAVENTURA,0654801,2023,Fase III - Diseños a detalle,Valle del Cauca,15.000,NaN,NaN,NaN,NaN,...,6.474906e+08,4.486878e+08,0.0,2.333073e+08,NaN,NaN,4.099869e+07,NaN,4.398490e+08,3.440089e+08
3,BUGA - BUENAVENTURA,0654801,2023,Fase III - Diseños a detalle,Valle del Cauca,3.120,NaN,NaN,1.0,NaN,...,6.474906e+08,4.486878e+08,0.0,2.333073e+08,NaN,NaN,4.099869e+07,NaN,4.398490e+08,3.440089e+08
4,BUGA - BUENAVENTURA,0654801,2023,Fase III - Diseños a detalle,Valle del Cauca,3.370,NaN,NaN,1.0,NaN,...,6.474906e+08,4.486878e+08,0.0,2.333073e+08,NaN,NaN,4.099869e+07,NaN,4.398490e+08,3.440089e+08
5,BUGA - BUENAVENTURA,0654801,2023,Fase III - Diseños a detalle,Valle del Cauca,0.800,NaN,NaN,NaN,NaN,...,6.474906e+08,4.486878e+08,0.0,2.333073e+08,NaN,NaN,4.099869e+07,NaN,4.398490e+08,3.440089e+08
6,BUGA - BUENAVENTURA,0654801,2023,Fase III - Diseños a detalle,Valle del Cauca,3.500,NaN,NaN,NaN,NaN,...,6.474906e+08,4.486878e+08,0.0,2.333073e+08,NaN,NaN,4.099869e+07,NaN,4.398490e+08,3.440089e+08
7,BUGA - BUENAVENTURA,0654801,2023,Fase III - Diseños a detalle,Valle del Cauca,5.170,NaN,NaN,1.0,NaN,...,6.474906e+08,4.486878e+08,0.0,2.333073e+08,NaN,NaN,4.099869e+07,NaN,4.398490e+08,3.440089e+08
8,BUGA - BUENAVENTURA,0654801,2023,Fase III - Diseños a detalle,Valle del Cauca,19.310,NaN,NaN,1.0,NaN,...,6.474906e+08,4.486878e+08,0.0,2.333073e+08,NaN,NaN,4.099869e+07,NaN,4.398490e+08,3.440089e+08
9,BUGA - BUENAVENTURA,0654801,2023,Fase III - Diseños a detalle,Valle del Cauca,8.210,NaN,NaN,NaN,NaN,...,6.474906e+08,4.486878e+08,0.0,2.333073e+08,NaN,NaN,4.099869e+07,NaN,4.398490e+08,3.440089e+08


In [5]:
df.columns

Index(['NOMBRE DEL PROYECTO', 'AÑO INICIO', 'LONGITUD KM',
       'PUENTES VEHICULARES UND', 'PUENTES VEHICULARES M2',
       'PUENTES PEATONALES UND', 'PUENTES PEATONALES M2', 'TUNELES UND',
       'TUNELES KM', 'ALCANCE', 'ZONA', 'TIPO TERRENO', 'NOMBRE UF',
       '1 - TRANSPORTE', '2 - TRAZADO Y DISEÑO GEOMÉTRICO',
       '2.1 - INFORMACIÓN GEOGRÁFICA', '2.2 TRAZADO Y DISEÑO GEOMÉTRICO',
       '2.3 - SEGURIDAD VIAL', '2.4 - SISTEMAS INTELIGENTES', '3 - GEOLOGÍA',
       '3.1 - GEOLOGÍA', '3.2 - HIDROGEOLOGÍA', '4 - SUELOS', '5 - TALUDES',
       '6 - PAVIMENTO', '7 - SOCAVACIÓN', '8 - ESTRUCTURAS', '9 - TÚNELES',
       '10 - URBANISMO Y PAISAJISMO', '11 - PREDIAL', '12 - IMPACTO AMBIENTAL',
       '13 - CANTIDADES', '14 - EVALUACIÓN SOCIOECONÓMICA',
       '15 - OTROS - MANEJO DE REDES', '16 - DIRECCIÓN Y COORDINACIÓN'],
      dtype='object')

In [5]:
import sqlite3
import sys
import pandas as pd
sys.path.append('..')
from src.config import Config

# Ensure DataFrame 'df' is available (use first cell's logic if needed)
try:
    df
except NameError:
    import eda
    filename = "../data/BASE DE DATOS PRESUPUESTOS.xlsx"
    preproccesing = eda.EDA(filename)
    df = preproccesing.assemble_project()

# Connect to database
db = sqlite3.connect(Config.DATABASE)
cursor = db.cursor()

# 1. Clean existing tables (delete all data)
print("Cleaning existing tables...")
cursor.execute("DELETE FROM item")
cursor.execute("DELETE FROM unidad_funcional")
cursor.execute("DELETE FROM proyectos")
db.commit()
print("✓ Tables cleaned")

# 2. Build projects dictionary
print("\nProcessing projects...")
proyectos_dict = {}

# Budget item columns are per-project totals; capture them once per project
item_columns = [
    '1 - TRANSPORTE',
    '2.1 - INFORMACIÓN GEOGRÁFICA', '2.2 TRAZADO Y DISEÑO GEOMÉTRICO',
    '2.3 - SEGURIDAD VIAL', '2.4 - SISTEMAS INTELIGENTES',
    '3.1 - GEOLOGÍA', '3.2 - HIDROGEOLOGÍA', '4 - SUELOS', '5 - TALUDES',
    '6 - PAVIMENTO', '7 - SOCAVACIÓN', '8 - ESTRUCTURAS', '9 - TÚNELES',
    '10 - URBANISMO Y PAISAJISMO', '11 - PREDIAL', '12 - IMPACTO AMBIENTAL',
    '13 - CANTIDADES', '14 - EVALUACIÓN SOCIOECONÓMICA',
    '15 - OTROS - MANEJO DE REDES', '16 - DIRECCIÓN Y COORDINACIÓN'
]

for idx, row in df.iterrows():
    codigo = str(row['CÓDIGO DEL PROYECTO']).strip()

    if codigo not in proyectos_dict:
        # First time seeing this project - store project-level data
        proyecto_items = {}
        costo_total = 0.0

        # Take item totals from the first row of each project to avoid double-counting
        for item_col in item_columns:
            if item_col in df.columns:
                valor = float(row[item_col]) if pd.notna(row[item_col]) else 0.0
                proyecto_items[item_col] = valor
                costo_total += valor

        proyectos_dict[codigo] = {
            'nombre': row['NOMBRE DEL PROYECTO'],
            'codigo': codigo,
            'anio_inicio': int(row['AÑO INICIO']) if pd.notna(row['AÑO INICIO']) else None,
            'fase': row['FASE'] if pd.notna(row['FASE']) else None,
            'ubicacion': row['DEPARTAMENTO'] if pd.notna(row['DEPARTAMENTO']) else None,
            'num_ufs': 0,
            'longitud': 0.0,
            'costo': round(costo_total, 2),
            'unidades': [],
            'items_totals': proyecto_items
        }

    # Track functional units and aggregate metrics
    proyecto = proyectos_dict[codigo]
    proyecto['num_ufs'] += 1
    proyecto['longitud'] += float(row['LONGITUD KM']) if pd.notna(row['LONGITUD KM']) else 0.0

    # Store functional unit data
    uf_data = {
        'unidad_funcional': proyecto['num_ufs'],  # Sequential number
        'longitud_km': float(row['LONGITUD KM']) if pd.notna(row['LONGITUD KM']) else 0.0,
        'puentes_vehiculares_und': int(row['PUENTES VEHICULARES UND']) if pd.notna(row['PUENTES VEHICULARES UND']) else 0,
        'puentes_vehiculares_mt2': int(row['PUENTES VEHICULARES M2']) if pd.notna(row['PUENTES VEHICULARES M2']) else 0,
        'puentes_peatonales_und': int(row['PUENTES PEATONALES UND']) if pd.notna(row['PUENTES PEATONALES UND']) else 0,
        'puentes_peatonales_mt2': int(row['PUENTES PEATONALES M2']) if pd.notna(row['PUENTES PEATONALES M2']) else 0,
        'tuneles_und': int(row['TUNELES UND']) if pd.notna(row['TUNELES UND']) else 0,
        'tuneles_km': float(row['TUNELES KM']) if pd.notna(row['TUNELES KM']) else 0.0,
        'alcance': row['ALCANCE'] if pd.notna(row['ALCANCE']) else None,
        'zona': row['ZONA'] if pd.notna(row['ZONA']) else None,
        'tipo_terreno': row['TIPO TERRENO'] if pd.notna(row['TIPO TERRENO']) else None,
    }
    proyectos_dict[codigo]['unidades'].append(uf_data)

print(f"✓ Found {len(proyectos_dict)} unique projects")

# 3. Insert projects
print("\nInserting projects into database...")
for codigo, proyecto in proyectos_dict.items():
    cursor.execute('''
        INSERT INTO proyectos (nombre, codigo, num_ufs, longitud, anio_inicio, duracion, fase, ubicacion, costo, lat_inicio, lng_inicio, lat_fin, lng_fin)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        proyecto['nombre'],
        proyecto['codigo'],
        proyecto['num_ufs'],
        round(proyecto['longitud'], 2),
        proyecto['anio_inicio'],
        None,  # duracion - not in dataframe
        proyecto['fase'],
        proyecto['ubicacion'],
        proyecto['costo'],
        None, None, None, None  # lat/lng - not in dataframe yet
    ))

db.commit()
print(f"✓ Inserted {len(proyectos_dict)} projects")

# 4. Insert functional units
print("\nInserting functional units...")
uf_count = 0
for codigo, proyecto in proyectos_dict.items():
    for uf in proyecto['unidades']:
        cursor.execute('''
            INSERT INTO unidad_funcional (codigo, unidad_funcional, longitud_km, puentes_vehiculares_und, puentes_vehiculares_mt2,
                                         puentes_peatonales_und, puentes_peatonales_mt2, tuneles_und, tuneles_km, alcance, zona, tipo_terreno)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            codigo,
            uf['unidad_funcional'],
            uf['longitud_km'],
            uf['puentes_vehiculares_und'],
            uf['puentes_vehiculares_mt2'],
            uf['puentes_peatonales_und'],
            uf['puentes_peatonales_mt2'],
            uf['tuneles_und'],
            uf['tuneles_km'],
            uf['alcance'],
            uf['zona'],
            uf['tipo_terreno']
        ))
        uf_count += 1

db.commit()
print(f"✓ Inserted {uf_count} functional units")

# 5. Insert items (aggregated per project; one-to-one with project)
print("\nInserting items...")
item_count = 0
for codigo, proyecto in proyectos_dict.items():
    for item_name, causado in proyecto['items_totals'].items():
        if causado > 0:  # Only insert non-zero items
            cursor.execute('''
                INSERT INTO item (codigo, item, causado)
                VALUES (?, ?, ?)
            ''', (codigo, item_name, round(causado, 2)))
            item_count += 1

db.commit()
print(f"✓ Inserted {item_count} items")

# Close connection
db.close()

print("\n" + "="*50)
print("DATABASE POPULATION COMPLETED SUCCESSFULLY!")
print("="*50)
print(f"Projects: {len(proyectos_dict)}")
print(f"Functional Units: {uf_count}")
print(f"Items: {item_count}")
print(f"Total Cost: ${sum(p['costo'] for p in proyectos_dict.values()):,.2f}")


Cleaning existing tables...
✓ Tables cleaned

Processing projects...
✓ Found 13 unique projects

Inserting projects into database...
✓ Inserted 13 projects

Inserting functional units...
✓ Inserted 51 functional units

Inserting items...
✓ Inserted 127 items

DATABASE POPULATION COMPLETED SUCCESSFULLY!
Projects: 13
Functional Units: 51
Items: 127
Total Cost: $11,259,686,115.60
